In [104]:
######################################################
# Juypter
######################################################
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))



################################################################################
## Music DB
################################################################################
from mainDB import mainDB
from musicDBMap import musicDBMap
from masterDBMatchClass import masterDBMatchClass
from matchDBArtist import matchDBArtist

from dbArtistsParseExtra import dbArtistsAssertExtra, dbArtistsExtra

######################################################
# Versions
######################################################
from timeUtils import clock, elapsed
import sys
print("Python: {0}".format(sys.version))
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2021-09-25 15:30:40.972125


In [103]:
#!python setup.py install

running install
running bdist_egg
running egg_info
writing dbdata.egg-info/PKG-INFO
writing dependency_links to dbdata.egg-info/dependency_links.txt
writing requirements to dbdata.egg-info/requires.txt
writing top-level names to dbdata.egg-info/top_level.txt
file artistAB.py (for module artistAB) not found
file artistCL.py (for module artistCL) not found
file artistMD.py (for module artistMD) not found
reading manifest file 'dbdata.egg-info/SOURCES.txt'
writing manifest file 'dbdata.egg-info/SOURCES.txt'
installing library code to build/bdist.macosx-10.9-x86_64/egg
running install_lib
running build_py
file artistAB.py (for module artistAB) not found
file artistCL.py (for module artistCL) not found
file artistMD.py (for module artistMD) not found
copying dbArtistsBase.py -> build/lib
copying dbArtistsParse.py -> build/lib
copying dbArtistsParseExtra.py -> build/lib
file artistAB.py (for module artistAB) not found
file artistCL.py (for module artistCL) not found
file artistMD.py (for mod

byte-compiling build/bdist.macosx-10.9-x86_64/egg/dbArtistsParse.py to dbArtistsParse.cpython-37.pyc
byte-compiling build/bdist.macosx-10.9-x86_64/egg/dbArtistsIHeart.py to dbArtistsIHeart.cpython-37.pyc
byte-compiling build/bdist.macosx-10.9-x86_64/egg/artistKWorbSpotify.py to artistKWorbSpotify.cpython-37.pyc
byte-compiling build/bdist.macosx-10.9-x86_64/egg/dbArtistsAllMusic.py to dbArtistsAllMusic.cpython-37.pyc
byte-compiling build/bdist.macosx-10.9-x86_64/egg/dbArtistsMusicStack.py to dbArtistsMusicStack.cpython-37.pyc
byte-compiling build/bdist.macosx-10.9-x86_64/egg/dbArtistsParseExtra.py to dbArtistsParseExtra.cpython-37.pyc
byte-compiling build/bdist.macosx-10.9-x86_64/egg/artistDP.py to artistDP.cpython-37.pyc
creating build/bdist.macosx-10.9-x86_64/egg/EGG-INFO
copying dbdata.egg-info/PKG-INFO -> build/bdist.macosx-10.9-x86_64/egg/EGG-INFO
copying dbdata.egg-info/SOURCES.txt -> build/bdist.macosx-10.9-x86_64/egg/EGG-INFO
copying dbdata.egg-info/dependency_links.txt -> build

Searching for prometheus-client==0.8.0
Best match: prometheus-client 0.8.0
Adding prometheus-client 0.8.0 to easy-install.pth file

Using /Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/site-packages
Searching for Jinja2==2.11.2
Best match: Jinja2 2.11.2
Adding Jinja2 2.11.2 to easy-install.pth file

Using /Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/site-packages
Searching for Send2Trash==1.5.0
Best match: Send2Trash 1.5.0
Adding Send2Trash 1.5.0 to easy-install.pth file

Using /Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/site-packages
Searching for ipykernel==5.3.0
Best match: ipykernel 5.3.0
Adding ipykernel 5.3.0 to easy-install.pth file

Using /Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/site-packages
Searching for pyzmq==18.1.1
Best match: pyzmq 18.1.1
Adding pyzmq 18.1.1 to easy-install.pth file

Using /Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/site-packages
Searching for terminado==0.8.3
Best match: terminado 0.8.3
Adding terminado 0.

# RateYourMusic

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsRateYourMusic import dbArtistsRateYourMusic
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertCredit, dbArtistsRawHTML

#dbAP = dbArtistsRawHTML(dbArtistsRateYourMusic())
#dbAP.parse(previousDays=1)

In [ ]:
%autoreload
for modVal in range(1):
    dbAP = dbArtistsPrimary(dbArtistsRateYourMusic())
    dometa = dbAP.parse(modVal, previousDays=None, force=True, debug=False)
    dometa = True
    continue
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

In [ ]:
%autoreload
from parseDBArtistsData import parseDBArtistsData
from dbArtistsParse import dbArtistsRawHTML
from dbArtistsRateYourMusic import dbArtistsRateYourMusic
pdb = parseDBArtistsData()
dbAP = dbArtistsRawHTML(dbArtistsRateYourMusic())
dbAP.parse(previousDays=7)

In [ ]:
from webUtils import getHTML
bsdata = getHTML("/Volumes/Piggy/Discog/artists-rateyourmusic/39/393739.p")

In [ ]:
%autoreload
dbAP = dbArtistsRateYourMusic(debug=True)
#retval = dbAP.artist.getData("/Volumes/Piggy/Discog/artists-rateyourmusic/39/393739.p")
retval = dbAP.artist.getData("/Volumes/Piggy/Discog/artists-rateyourmusic/5/2605.p")
#retval = dbAP.artist.getData("/Volumes/Piggy/Discog/artists-rateyourmusic/72/80772.p")
#retval = dbAP.artist.getData("/Volumes/Piggy/Discog/artists-rateyourmusic/10/7410.p")

In [ ]:
retval.profile.currently.get()

In [ ]:
[x.get() for x in retval.profile.genres]

In [ ]:
%autoreload
from dbArtistsRateYourMusic import dbArtistsRateYourMusic
from artistRateYourMusic import artistRateYourMusic
dbAP = dbArtistsRateYourMusic(debug=True)
retval = dbAP.artist.getData("/Volumes/Piggy/Discog/artists-rateyourmusic/5/2605.p")

In [ ]:
retval.show()

In [ ]:
retval.profile.relatedartists.get()

In [ ]:
retval.show()

In [ ]:
retval.profile.members[0].get()

In [ ]:
ap = artistRateYourMusic(debug=True)

In [ ]:
%autoreload
from artistDBBase import artistDBLinkClass, artistDBTagClass
from artistRateYourMusic import artistRateYourMusic
from webUtils import getHTML
bsdata = getHTML("/Volumes/Piggy/Discog/artists-rateyourmusic/56/56.p")


In [ ]:
profileData.keys()

In [ ]:
profileData["Notes"]

In [ ]:
retval = ap.getData("/Volumes/Piggy/Discog/artists-rateyourmusic/56/56.p")
retval.show()

In [ ]:
retval = dbAP.artist.getData()

In [ ]:
retval.show()

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsRateYourMusic())
    dometa = dbAP.parse(modVal, previousDays=2, force=False)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

# Genius

In [ ]:

#dometa = dbAP.parse(0, previousDays=None, force=True)

In [ ]:
from ioUtils import getFile
data = getFile("/Users/tgadfort/Music/Discog/artists-genius-db/0-DB.p")

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsGenius import dbArtistsGenius
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial, dbArtistsAssertCredit
modVals=range(39,40)
dbAP = dbArtistsPrimary(dbArtistsGenius())
from searchUtils import findExt
from fileUtils import getDirname, getBaseFilename
from fsUtils import setFile, moveFile, isFile, removeFile, setDir
from dbUtils import utilsGenius
utils = utilsGenius()
for modVal in modVals:
    files = findExt("/Volumes/Piggy/Discog/artists-genius/{0}/".format(modVal), ext=".p")
    print(modVal,len(files))
    for ifile in files:
        print('\t',ifile,'\t',end="")
        try:
            retval = dbAP.artist.getData(ifile)
        except:
            print("Bad File: {0}".format(ifile))
            removeFile(ifile)
            continue
        artistID = retval.get()["ID"]
        artistID = artistID.ID
        print(artistID,'\t',end="")
        
        artistURL = retval.get()["url"]
        artistURL = artistURL.url
        artistIDFromURL = utils.getArtistID(artistURL)
        print(artistIDFromURL,'\t',artistURL)
        
        try:
            int(artistID)
        except:
            print("Bad File: {0}".format(ifile))
            removeFile(ifile)
            continue
        #print(getDirname(ifile),getBaseFilename(ifile))
        srcFile = ifile
        dirval  = getDirname(getDirname(ifile))
        modval  = str(int(artistID) % 100)
        
        dstFile = setFile(setDir(dirval, modval), "{0}.p".format(artistID))
        if srcFile == dstFile:
            continue
        if isFile(dstFile):
            print("{0} already exists.".format(dstFile))
            retval2 = dbAP.artist.getData(dstFile)
            artistID2 = retval2.get()["ID"]
            artistID2 = artistID2.ID
            if artistID2 == artistID:
                print("\tRemove old file: {0}".format(ifile))
                removeFile(ifile)
            continue
            
            raise ValueError(srcFile,dstFile)
        print(srcFile,dstFile)
        moveFile(srcFile, dstFile)

In [ ]:
dbAP = dbArtistsPrimary(dbArtistsGenius())
dometa = dbAP.parse(0, previousDays=None, force=True, debug=True)

In [ ]:
ifile='/Volumes/Piggy/Discog/artists-genius/2/83329186.p'
retval = dbAP.artist.getData(ifile)
retval.show()

In [ ]:
ifile='/Volumes/Piggy/Discog/artists-genius/63/82287463.p'
retval = dbAP.artist.getData(ifile)
retval.show()

In [ ]:
dbAP.dutils.getArtistID('https://genius.com/artists/The-arcana', debug=True)

In [ ]:
#8606107
from webUtils import getHTML
ifile = getHTML("/Volumes/Piggy/Discog/artists-genius/0/49153000.p")
ifile

In [ ]:
retval = dbAP.artist.getData(ifile)
retval.show()

In [ ]:
data.keys()

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsGenius import dbArtistsGenius
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial, dbArtistsAssertCredit

In [ ]:
dbAP = dbArtistsPrimary(dbArtistsGenius())
dometa = dbAP.parse(56, previousDays=None, force=False, debug=True)

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsGenius())
    dometa = dbAP.parse(modVal, previousDays=None, force=True, debug=False)
    dometa = True
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

# AlbumOfTheYear

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsAlbumOfTheYear import dbArtistsAlbumOfTheYear
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsAlbumOfTheYear())
    dometa = dbAP.parse(modVal, previousDays=10, force=False)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

# Deezer

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsDeezer import dbArtistsDeezer
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial

In [ ]:
dbAP = dbArtistsPrimary(dbArtistsDeezer())
dometa = dbAP.parse(44, previousDays=1, force=True)
if dometa:
    dbAP.createArtistMetadata(44)
    dbAP.createAlbumMetadata(44)

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsDeezer import dbArtistsDeezer
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial
dbAP = dbArtistsDeezer(debug=True)

# MusicBrainz

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsMusicBrainz import dbArtistsMusicBrainz
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial
dbMB = dbArtistsMusicBrainz()
for modVal in range(2,100):
    #if True:
    #try:
        #lfm.assertDBModValExtraData(modVal, minPages=10, maxPages=20, test=False)
    dbMB.assertDBModValExtraData(modVal, minPages=2, maxPages=50, test=False, clean=True)
    #except:
    #    pass

# AllMusic

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsAllMusic import dbArtistsAllMusic
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial
from dbArtistsParse import dbArtistsAssertCredit, dbArtistsAssertComposition, dbArtistsAssertSong
dbAP = dbArtistsAllMusic(debug=True)

In [ ]:
artistDBData = dbAP.disc.getArtistsDBModValData(0)

In [ ]:
artistDBData['0000006000'].show()

In [ ]:
for artistID,artistData in artistDBData.items():
    print(artistID,'\t',artistData.media.media.keys())

In [ ]:
dbAP = dbArtistsAssertComposition(dbArtistsAllMusic())
dbAP.parseModVal(0)

In [ ]:
from ioUtils import getFile
data = getFile("/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/0-Metadata.p")

In [ ]:
{data

In [ ]:
dbAP = dbArtistsTest(dbArtistsAllMusic())
dbAP.parseModVal(0, force=True)

In [ ]:
suburl='tv-smiths-explorers-mn0000008802/discography'
if suburl.endswith("/discography"):
    suburl=suburl[:-12]

In [ ]:
dbAP = dbArtistsPrimary(dbArtistsAllMusic())
dometa = dbAP.parse(2, previousDays=None, force=True, debug=False)

In [ ]:
dbac = dbArtistsAssertCredit(dbArtistsAllMusic())
print(dbac.ignores.keys())

In [ ]:
dbAP = dbArtistsAssertCredit(dbArtistsAllMusic())
dbAP.parse(2, previousDays=None)

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsAllMusic())
    dometa = dbAP.parse(modVal, previousDays=60, force=False)
    dbAP.createArtistMetadata(modVal)
    dbAP.createAlbumMetadata(modVal)

In [ ]:
dbac = dbArtistsAssertComposition(dbArtistsAllMusic())
dbac.parseModVal(12)
dbac.downloadUnknownArtistCompositions()

In [ ]:
# This downloads missing credit files
for modVal in range(70,100):
    print(modVal)
    try:
        dbac = dbArtistsAssertCredit(dbArtistsAllMusic())
        dbac.parseModVal(modVal)
        dbac.downloadUnknownArtistCredits()
    except:
        continue


In [ ]:
%autoreload
from parseDBArtistsData import parseDBArtistsData
from dbArtistsParse import dbArtistsPrimary
from dbArtistsRateYourMusic import dbArtistsRateYourMusic

for modVal in range(1):
    dbAP = dbArtistsPrimary(dbArtistsRateYourMusic())
    dometa = dbAP.parse(modVal, previousDays=None, force=True, debug=False)
    #if dometa and False:
    #    dbAP.createArtistMetadata(modVal)
    #    dbAP.createAlbumMetadata(modVal)

In [ ]:


for modVal in range(100):
    dbAP = dbArtistsCredit(dbArtistsAllMusic())
    dbAP.parse(modVal, previousDays=30)
    

for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsAllMusic())
    dometa = dbAP.parse(modVal, previousDays=None, force=False)
    if dometa or True:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

In [ ]:
from ioUtils import getFile
x = getFile("/Users/tgadfort/Music/Discog/artists-deezer-db/metadata/5-MediaMetadata.p")

In [ ]:
x['5038705']

In [ ]:
modVal = 5
dbAP = dbArtistsPrimary(dbArtistsDeezer())
dometa = dbAP.parse(modVal, previousDays=1, force=True)
if dometa:
    dbAP.createArtistMetadata(modVal)
    dbAP.createAlbumMetadata(modVal)

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsDeezer())
    dometa = dbAP.parse(modVal, previousDays=1, force=True)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

# Discogs

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsDiscogs import dbArtistsDiscogs
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsDiscogs())
    dometa = dbAP.parse(modVal, previousDays=3, force=False)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsAssertUnofficial(dbArtistsDiscogs())
    dbAP.parseModVal(modVal)
    dbAP.downloadMissingArtistUnofficials()

# MusicBrainz

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsMusicBrainz import dbArtistsMusicBrainz
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsMusicBrainz())
    dometa = dbAP.parse(modVal, previousDays=10, force=False)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

#for modVal in range(100):
#    dbAP = dbArtistsExtra(dbArtistsMusicBrainz())
#    dbAP.parse(modVal)

# LastFM

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsLastFM import dbArtistsLastFM
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsLastFM())
    dometa = dbAP.parse(modVal, previousDays=10, force=False)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

# KWorbSpotify

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsKWorbSpotify import dbArtistsKWorbSpotify
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial, dbArtistsRawFiles
from time import sleep

#dbAP = dbArtistsRawFiles(dbArtistsKWorbSpotify(), "data")
#dbAP.parse(previousDays=1000000)

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsKWorbSpotify())
    dometa = dbAP.parse(modVal, previousDays=3000, force=True, debug=False)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)
        
sleep(1)
start, cmt = clock("Creating DB")
maindb = mainDB(mdb=None, create=True, debug=True)
maindb.setDBFull("KWorbSpotify")
elapsed(start, cmt)    

# KWorbiTunes

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsKWorbiTunes import dbArtistsKWorbiTunes
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial, dbArtistsRawFiles
from time import sleep

#dbAP = dbArtistsRawFiles(dbArtistsKWorbiTunes(), "data")
#dbAP.parse(previousDays=1000000)

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsKWorbiTunes())
    dometa = dbAP.parse(modVal, previousDays=3000, force=True, debug=False)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)
        
sleep(1)
start, cmt = clock("Creating DB")
maindb = mainDB(mdb=None, create=True, debug=True)
maindb.setDBFull("KWorbiTunes")
elapsed(start, cmt)    

# KWorbYouTube

In [ ]:
# %autoreload
from dbArtistsKWorbYouTube import dbArtistsKWorbYouTube
dbAP = dbArtistsRawFiles(dbArtistsKWorbYouTube(), "data")
dbAP.parse(previousDays=10)

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsKWorbYouTube import dbArtistsKWorbYouTube
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial, dbArtistsRawFiles
from time import sleep

dbAP = dbArtistsRawFiles(dbArtistsKWorbYouTube(), "data")
#dbAP.parse(previousDays=1000000)

for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsKWorbYouTube())
    dometa = dbAP.parse(modVal, previousDays=3, force=True)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)
        
sleep(1)
start, cmt = clock("Creating DB")
maindb = mainDB(mdb=None, create=True, debug=True)
maindb.setDBFull("KWorbYouTube")
elapsed(start, cmt)    

In [ ]:
%autoreload
from dbArtistsRateYourMusic import dbArtistsRateYourMusic
#pdb = parseDBArtistsData()
dbAP = dbArtistsRawHTML(dbArtistsRateYourMusic())
dbAP.parse('< 5 days')

In [ ]:
%autoreload
from dbArtistsRateYourMusic import dbArtistsRateYourMusic
#pdb = parseDBArtistsData()
dbAP = dbArtistsPrimary(dbArtistsRateYourMusic())
for modVal in range(100):
    dbAP.parse(modVal=modVal, expr='< 0 hours')

## Assert Credit For AllMusic

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsAllMusic import dbArtistsAllMusic
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial, dbArtistsAssertCredit
dbAP = dbArtistsAllMusic(debug=True)
# This downloads missing credit files
ts = timestat("Getting Credit Files")
for modVal in range(20,30):
    try:
        dbac = dbArtistsAssertCredit(dbArtistsAllMusic())
        dbac.parseModVal(modVal)
        dbac.downloadUnknownArtistCredits()
    except:
        continue
        
ts.stop()

In [24]:
%load_ext autoreload
%autoreload
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra, dbArtistsAssertCredit
from dbArtistsAllMusic import dbArtistsAllMusic

modVal=86
dbAC = dbArtistsAssertCredit(dbArtistsAllMusic())
dbAC.createCreditMetadata(modVal=modVal)
dbAC.downloadUnknownArtistCredits()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
=========================   allmusic   =========================
dbArtistsAssertCredit(AllMusic)
  --> Found 68 AllMusic Credit IDs To Ignore
Current Time is Fri Sep 24, 2021 14:09 for Creating AllMusic Credit Metadata
Current Time is Fri Sep 24, 2021 14:09 for Finding Primary Files For ModVal=86
Process [Finding Primary Files For ModVal=86] took 0.7 seconds.
Current Time is Fri Sep 24, 2021 14:09 for Removing IDs To Ignore From 7236 Primary Files For ModVal=7236
Process [Removing IDs To Ignore From 7236 Primary Files For ModVal=7236] took 0.0 seconds.
Current Time is Fri Sep 24, 2021 14:09 for Finding Known Artists From 7236 Primary/Good Files For ModVal=86
Process [Finding Known Artists From 7236 Primary/Good Files For ModVal=86] took 3.3 seconds.
Current Time is Fri Sep 24, 2021 14:09 for Finding Known Credit Artists From 1183 Unknown Artists For ModVal=86
Process [Finding Known Credit Artists Fr

Saving /Volumes/Piggy/Discog/artists-allmusic/86/credit/0000067286.p (force=False)
Done. Sleeping for 2 seconds
15/1183:  [boo! hiss! pfftlb!]


===================== Searching For boo! hiss! pfftlb! , 0000081486 =====================
1 / 2 	: 0000081486 	 https://www.allmusic.com/artist/boo%21-hiss%21-pfftlb%21-mn0000081486/credits
URL/Savename: [https://www.allmusic.com/artist/boo%21-hiss%21-pfftlb%21-mn0000081486/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/86/credit/0000081486.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/86/credit/0000081486.p (force=False)
Done. Sleeping for 2 seconds
16/1183:  [cash]


===================== Searching For cash , 0000087686 =====================
1 / 19 	: 0000087686 	 https://www.allmusic.com/artist/cash-mn0000087686/credits
URL/Savename: [https://www.allmusic.com/artist/cash-mn0000087686/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/86/credit/0000087686.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/86/credit/0000087686.p (force

Saving /Volumes/Piggy/Discog/artists-allmusic/86/credit/0000157486.p (force=False)
Done. Sleeping for 2 seconds
33/1183:  [kenny jackel]


===================== Searching For kenny jackel , 0000159186 =====================
1 / 20 	: 0000159186 	 https://www.allmusic.com/artist/kenny-jackel-mn0000159186/credits
URL/Savename: [https://www.allmusic.com/artist/kenny-jackel-mn0000159186/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/86/credit/0000159186.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/86/credit/0000159186.p (force=False)
Done. Sleeping for 2 seconds
34/1183:  [richard evans]


===================== Searching For richard evans , 0000162086 =====================
1 / 20 	: 0000162086 	 https://www.allmusic.com/artist/richard-evans-mn0000162086/credits
URL/Savename: [https://www.allmusic.com/artist/richard-evans-mn0000162086/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/86/credit/0000162086.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/86/credit/0000162086.p (forc

Saving /Volumes/Piggy/Discog/artists-allmusic/86/credit/0000249386.p (force=False)
Done. Sleeping for 2 seconds
51/1183:  [mn000025068]


===================== Searching For mn000025068 , 0000250686 =====================
52/1183:  [jon walls]


===================== Searching For jon walls , 0000259386 =====================
1 / 20 	: 0000259386 	 https://www.allmusic.com/artist/jon-walls-mn0000259386/credits
URL/Savename: [https://www.allmusic.com/artist/jon-walls-mn0000259386/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/86/credit/0000259386.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/86/credit/0000259386.p (force=False)
Done. Sleeping for 2 seconds
53/1183:  [Jeremy Kipnis]


===================== Searching For Jeremy Kipnis , 0000270586 =====================
1 / 18 	: 0000270586 	 https://www.allmusic.com/artist/jeremy-kipnis-mn0000270586/credits
URL/Savename: [https://www.allmusic.com/artist/jeremy-kipnis-mn0000270586/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/8

Saving /Volumes/Piggy/Discog/artists-allmusic/86/credit/0000330486.p (force=False)
Done. Sleeping for 2 seconds
71/1183:  [Mark Vernon]


===================== Searching For Mark Vernon , 0000334786 =====================
1 / 20 	: 0000334786 	 https://www.allmusic.com/artist/mark-vernon-mn0000334786/credits
URL/Savename: [https://www.allmusic.com/artist/mark-vernon-mn0000334786/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/86/credit/0000334786.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/86/credit/0000334786.p (force=False)
Done. Sleeping for 2 seconds
72/1183:  [jerry seelen]


===================== Searching For jerry seelen , 0000334886 =====================
1 / 20 	: 0000334886 	 https://www.allmusic.com/artist/jerry-seelen-mn0000334886/credits
URL/Savename: [https://www.allmusic.com/artist/jerry-seelen-mn0000334886/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/86/credit/0000334886.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/86/credit/0000334886.p (force=False)

...

Saving /Volumes/Piggy/Discog/artists-allmusic/86/credit/0003736786.p (force=False)
Done. Sleeping for 2 seconds
1108/1183:  [david frank]


===================== Searching For david frank , 0003738786 =====================
2 / 20 	: 0003738786 	 https://www.allmusic.com/artist/david-frank-mn0003738786/credits
URL/Savename: [https://www.allmusic.com/artist/david-frank-mn0003738786/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/86/credit/0003738786.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/86/credit/0003738786.p (force=False)
Done. Sleeping for 2 seconds
1109/1183:  [john petri]


===================== Searching For john petri , 0003740186 =====================
1 / 20 	: 0003740186 	 https://www.allmusic.com/artist/john-petri-mn0003740186/credits
URL/Savename: [https://www.allmusic.com/artist/john-petri-mn0003740186/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/86/credit/0003740186.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/86/credit/0003740186.p (force=False)
Don

Saving /Volumes/Piggy/Discog/artists-allmusic/86/credit/0003790286.p (force=False)
Done. Sleeping for 2 seconds
1126/1183:  [julio dias de castro]


===================== Searching For julio dias de castro , 0003794186 =====================
1 / 2 	: 0003794186 	 https://www.allmusic.com/artist/julio-dias-de-castro-mn0003794186/credits
URL/Savename: [https://www.allmusic.com/artist/julio-dias-de-castro-mn0003794186/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/86/credit/0003794186.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/86/credit/0003794186.p (force=False)
Done. Sleeping for 2 seconds
1127/1183:  [giuseppe torrebruno]


===================== Searching For giuseppe torrebruno , 0003796686 =====================
1 / 19 	: 0003796686 	 https://www.allmusic.com/artist/giuseppe-torrebruno-mn0003796686/credits
URL/Savename: [https://www.allmusic.com/artist/giuseppe-torrebruno-mn0003796686/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/86/credit/0003796686.p]
Saving /Volumes

1 / 6 	: 0003875886 	 https://www.allmusic.com/artist/neville-hinds-the-rupie-edwards-all-stars-mn0003875886/credits
URL/Savename: [https://www.allmusic.com/artist/neville-hinds-the-rupie-edwards-all-stars-mn0003875886/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/86/credit/0003875886.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/86/credit/0003875886.p (force=False)
Done. Sleeping for 2 seconds
1144/1183:  [drum dummie]


===================== Searching For drum dummie , 0003886886 =====================
1 / 19 	: 0003886886 	 https://www.allmusic.com/artist/drum-dummie-mn0003886886/credits
URL/Savename: [https://www.allmusic.com/artist/drum-dummie-mn0003886886/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/86/credit/0003886886.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/86/credit/0003886886.p (force=False)
Done. Sleeping for 2 seconds
1145/1183:  [ricardo cardenas alvarez]


===================== Searching For ricardo cardenas alvarez , 0003909186 ===================

Saving /Volumes/Piggy/Discog/artists-allmusic/86/credit/0004004886.p (force=False)
Done. Sleeping for 2 seconds
1161/1183:  [nc]


===================== Searching For nc , 0004012886 =====================
3 / 20 	: 0004012886 	 https://www.allmusic.com/artist/nc-mn0004012886/credits
URL/Savename: [https://www.allmusic.com/artist/nc-mn0004012886/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/86/credit/0004012886.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/86/credit/0004012886.p (force=False)
Done. Sleeping for 2 seconds
1162/1183:  [jahn bonfiglio]


===================== Searching For jahn bonfiglio , 0004018686 =====================
1 / 20 	: 0004018686 	 https://www.allmusic.com/artist/jahn-bonfiglio-mn0004018686/credits
URL/Savename: [https://www.allmusic.com/artist/jahn-bonfiglio-mn0004018686/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/86/credit/0004018686.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/86/credit/0004018686.p (force=False)
Done. Sleeping for 2 se

1179/1183:  [william white]


===================== Searching For william white , 0004122786 =====================
14 / 20 	: 0004122786 	 https://www.allmusic.com/artist/william-white-mn0004122786/credits
URL/Savename: [https://www.allmusic.com/artist/william-white-mn0004122786/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/86/credit/0004122786.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/86/credit/0004122786.p (force=False)
Done. Sleeping for 2 seconds
1180/1183:  [eva ibanez fraile]


===================== Searching For eva ibanez fraile , 0004123186 =====================
1 / 2 	: 0004123186 	 https://www.allmusic.com/artist/eva-ibanez-fraile-mn0004123186/credits
URL/Savename: [https://www.allmusic.com/artist/eva-ibanez-fraile-mn0004123186/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/86/credit/0004123186.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/86/credit/0004123186.p (force=False)
Done. Sleeping for 2 seconds
1181/1183:  [morrell patterson]


================

In [26]:
dbC = dbArtistsCredit(dbArtistsAllMusic())
#dbC.parse(modVal=97, expr='< 90 Days', force=False)
dbC.parse(modVal=97, expr=None, force=False)

=========================   allmusic   =========================
Current Time is Fri Sep 24, 2021 15:34 for Parsing ModVal=97 Credit Files
Current Time is Fri Sep 24, 2021 15:34 for Finding Files To Parse


AttributeError: 'NoneType' object has no attribute 'lower'

In [27]:
%load_ext autoreload
%autoreload
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra, dbArtistsAssertCredit
from dbArtistsAllMusic import dbArtistsAllMusic

for modVal in range(80,81):
    dbAP = dbArtistsPrimary(dbArtistsAllMusic())
    _ = dbAP.parse(modVal=modVal, expr='< 0 Days', force=False)
    dbAP.createArtistMetadata(modVal)
    dbAP.createAlbumMetadata(modVal)
    
    dbAC = dbArtistsAssertCredit(dbArtistsAllMusic())
    dbAC.createCreditMetadata(modVal=modVal)
    dbAC.downloadUnknownArtistCredits()
    
    dbC = dbArtistsCredit(dbArtistsAllMusic())
    dbC.parse(modVal=modVal, expr='< 90 Days', force=False)
    dbAP.createArtistMetadata(modVal)
    dbAP.createAlbumMetadata(modVal)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
=========================   allmusic   =========================
Current Time is Fri Sep 24, 2021 15:34 for Parsing ModVal=80 Primary Files
Current Time is Fri Sep 24, 2021 15:34 for Finding Files To Parse
Process [Finding Files To Parse] took 1.5 minutes.
Current Time is Fri Sep 24, 2021 15:36 for Loading ModVal=80 DB Data
Process [Loading ModVal=80 DB Data] took 3.8 seconds.
Current Time is Fri Sep 24, 2021 15:36 for Parsing 82 New Files For ModVal=80
50/82          Parsing /Volumes/Piggy/Discog/artists-allmusic/80/0001896180.p
82/82          Parsing /Volumes/Piggy/Discog/artists-allmusic/80/0003859980.p
Process [Parsing 82 New Files For ModVal=80] took 4.4 seconds.
Saving 82 new artist IDs to /Users/tgadfort/Music/Discog/artists-allmusic-db/80-DB.p
Saving 6174 total artist IDs to /Users/tgadfort/Music/Discog/artists-allmusic-db/80-DB.p
Saving 47601 total artist media
Process [Parsing ModVal=80 Pr

1 / 19 	: 0000086280 	 https://www.allmusic.com/artist/kent-robbins-mn0000086280/credits
URL/Savename: [https://www.allmusic.com/artist/kent-robbins-mn0000086280/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/80/credit/0000086280.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/80/credit/0000086280.p (force=False)
Done. Sleeping for 2 seconds
13/1022:  [andy haderer]


===================== Searching For andy haderer , 0000089080 =====================
1 / 20 	: 0000089080 	 https://www.allmusic.com/artist/andy-haderer-mn0000089080/credits
URL/Savename: [https://www.allmusic.com/artist/andy-haderer-mn0000089080/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/80/credit/0000089080.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/80/credit/0000089080.p (force=False)
Done. Sleeping for 2 seconds
14/1022:  [james straud]


===================== Searching For james straud , 0000097580 =====================
1 / 20 	: 0000097580 	 https://www.allmusic.com/artist/james-straud-mn00000975

2 / 20 	: 0000177180 	 https://www.allmusic.com/artist/billy-bowen-mn0000177180/credits
URL/Savename: [https://www.allmusic.com/artist/billy-bowen-mn0000177180/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/80/credit/0000177180.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/80/credit/0000177180.p (force=False)
Done. Sleeping for 2 seconds
31/1022:  [jan koster]


===================== Searching For jan koster , 0000177580 =====================
1 / 20 	: 0000177580 	 https://www.allmusic.com/artist/jan-koster-mn0000177580/credits
URL/Savename: [https://www.allmusic.com/artist/jan-koster-mn0000177580/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/80/credit/0000177580.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/80/credit/0000177580.p (force=False)
Done. Sleeping for 2 seconds
32/1022:  [enos barnes]


===================== Searching For enos barnes , 0000180080 =====================
1 / 19 	: 0000180080 	 https://www.allmusic.com/artist/enos-barnes-mn0000180080/credits
UR

48/1022:  [lou whitney]


===================== Searching For lou whitney , 0000233980 =====================
1 / 20 	: 0000233980 	 https://www.allmusic.com/artist/lou-whitney-mn0000233980/credits
URL/Savename: [https://www.allmusic.com/artist/lou-whitney-mn0000233980/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/80/credit/0000233980.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/80/credit/0000233980.p (force=False)
Done. Sleeping for 2 seconds
49/1022:  [davod hahn]


===================== Searching For davod hahn , 0000234280 =====================
1 / 20 	: 0000234280 	 https://www.allmusic.com/artist/davod-hahn-mn0000234280/credits
URL/Savename: [https://www.allmusic.com/artist/davod-hahn-mn0000234280/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/80/credit/0000234280.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/80/credit/0000234280.p (force=False)
Done. Sleeping for 2 seconds
50/1022:  [roberto pulido y los classicos]


===================== Searching For roberto p

67/1022:  [katherine miller]


===================== Searching For katherine miller , 0000367580 =====================
1 / 20 	: 0000367580 	 https://www.allmusic.com/artist/katherine-miller-mn0000367580/credits
URL/Savename: [https://www.allmusic.com/artist/katherine-miller-mn0000367580/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/80/credit/0000367580.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/80/credit/0000367580.p (force=False)
Done. Sleeping for 2 seconds
68/1022:  [patrick arena and andy monroe]


===================== Searching For patrick arena and andy monroe , 0000372880 =====================
1 / 1 	: 0000372880 	 https://www.allmusic.com/artist/patrick-arena-and-andy-monroe-mn0000372880/credits
URL/Savename: [https://www.allmusic.com/artist/patrick-arena-and-andy-monroe-mn0000372880/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/80/credit/0000372880.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/80/credit/0000372880.p (force=False)
Done. Sleeping for 2 sec

...

Saving /Volumes/Piggy/Discog/artists-allmusic/80/credit/0003823280.p (force=False)
Done. Sleeping for 2 seconds
944/1022:  [ricki lamar thomas]


===================== Searching For ricki lamar thomas , 0003823880 =====================
1 / 4 	: 0003823880 	 https://www.allmusic.com/artist/ricki-lamar-thomas-mn0003823880/credits
URL/Savename: [https://www.allmusic.com/artist/ricki-lamar-thomas-mn0003823880/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/80/credit/0003823880.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/80/credit/0003823880.p (force=False)
Done. Sleeping for 2 seconds
945/1022:  [yung dreamy]


===================== Searching For yung dreamy , 0003824780 =====================
1 / 20 	: 0003824780 	 https://www.allmusic.com/artist/yung-dreamy-mn0003824780/credits
URL/Savename: [https://www.allmusic.com/artist/yung-dreamy-mn0003824780/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/80/credit/0003824780.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/80/credit/0

5 / 20 	: 0003903380 	 https://www.allmusic.com/artist/nicholas-martin-mn0003903380/credits
URL/Savename: [https://www.allmusic.com/artist/nicholas-martin-mn0003903380/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/80/credit/0003903380.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/80/credit/0003903380.p (force=False)
Done. Sleeping for 2 seconds
962/1022:  [booze willis]


===================== Searching For booze willis , 0003907880 =====================
1 / 19 	: 0003907880 	 https://www.allmusic.com/artist/booze-willis-mn0003907880/credits
URL/Savename: [https://www.allmusic.com/artist/booze-willis-mn0003907880/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/80/credit/0003907880.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/80/credit/0003907880.p (force=False)
Done. Sleeping for 2 seconds
963/1022:  [drez william]


===================== Searching For drez william , 0003908180 =====================
1 / 20 	: 0003908180 	 https://www.allmusic.com/artist/drez-william-mn

979/1022:  [maria de lourdes torres]


===================== Searching For maria de lourdes torres , 0003965780 =====================
1 / 8 	: 0003965780 	 https://www.allmusic.com/artist/maria-de-lourdes-torres-mn0003965780/credits
URL/Savename: [https://www.allmusic.com/artist/maria-de-lourdes-torres-mn0003965780/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/80/credit/0003965780.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/80/credit/0003965780.p (force=False)
Done. Sleeping for 2 seconds
980/1022:  [cani marino]


===================== Searching For cani marino , 0003968680 =====================
1 / 20 	: 0003968680 	 https://www.allmusic.com/artist/cani-marino-mn0003968680/credits
URL/Savename: [https://www.allmusic.com/artist/cani-marino-mn0003968680/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/80/credit/0003968680.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/80/credit/0003968680.p (force=False)
Done. Sleeping for 2 seconds
981/1022:  [tim mariza]


===========

998/1022:  [luri lodovichi]


===================== Searching For luri lodovichi , 0004048680 =====================
1 / 12 	: 0004048680 	 https://www.allmusic.com/artist/luri-lodovichi-mn0004048680/credits
URL/Savename: [https://www.allmusic.com/artist/luri-lodovichi-mn0004048680/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/80/credit/0004048680.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/80/credit/0004048680.p (force=False)
Done. Sleeping for 2 seconds
999/1022:  [darrell r bentley]


===================== Searching For darrell r bentley , 0004048780 =====================
1 / 2 	: 0004048780 	 https://www.allmusic.com/artist/darrell-r-bentley-mn0004048780/credits
URL/Savename: [https://www.allmusic.com/artist/darrell-r-bentley-mn0004048780/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/80/credit/0004048780.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/80/credit/0004048780.p (force=False)
Done. Sleeping for 2 seconds
1000/1022:  [nord nord muzikk]


================

1016/1022:  [alireza alavian]


===================== Searching For alireza alavian , 0004114880 =====================
1 / 19 	: 0004114880 	 https://www.allmusic.com/artist/alireza-alavian-mn0004114880/credits
URL/Savename: [https://www.allmusic.com/artist/alireza-alavian-mn0004114880/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/80/credit/0004114880.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/80/credit/0004114880.p (force=False)
Done. Sleeping for 2 seconds
1017/1022:  [allison v brown]


===================== Searching For allison v brown , 0004117980 =====================
1 / 7 	: 0004117980 	 https://www.allmusic.com/artist/allison-v-brown-mn0004117980/credits
URL/Savename: [https://www.allmusic.com/artist/allison-v-brown-mn0004117980/credits] :: [/Volumes/Piggy/Discog/artists-allmusic/80/credit/0004117980.p]
Saving /Volumes/Piggy/Discog/artists-allmusic/80/credit/0004117980.p (force=False)
Done. Sleeping for 2 seconds
1018/1022:  [amit amty takkar]


==================

In [ ]:
artistIDs=['0000047990', '0000051390', '0000272190', '0000292790', '0000467590', '0000602690', '0000717990', '0000796690', '0001033390', '0001378390', '0001407190', '0001464290', '0001995390', '0002323590', '0002425790', '0002765190', '0002879590', '0002903990', '0003093090', '0003190590', '0003246290', '0003286290', '0003918290']
artistIDs+=['0000118299', '0000213799', '0000325299', '0000642999', '0000775799', '0000811999', '0000992099', '0001284999', '0001364399', '0001497499', '0001510299', '0001618699', '0001634699', '0001658799', '0001694699', '0001711999', '0002179199', '0002366199', '0002612899', '0002752799', '0002987499', '0003027599', '0003184899', '0003258299', '0003442899', '0003678399']
len(artistIDs)

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsAllMusic import dbArtistsAllMusic
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial, dbArtistsAssertCredit
tmp = dbArtistsAssertCredit(dbArtistsAllMusic())
#tmp.updateMasterIgnoreCreditData(artistIDs)
#tmp.createCreditMetadata(modVal=98)
#tmp.downloadUnknownArtistCredits()

In [ ]:
from copy import deepcopy
meta = deepcopy(tmp.metadata)

In [ ]:
tmp.downloadUnknownArtistCredits()

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsAllMusic import dbArtistsAllMusic
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial, dbArtistsAssertCredit
tmp = dbArtistsCredit(dbArtistsAllMusic())
tmp.parse(modVal=99, expr='< 90 Days', force=False)

In [ ]:
from searchUtils import filesFromDir
from fsUtils import setDir
force=True
modVal=99
ffd   = filesFromDir(ext=".p")
files = ffd.getFiles(setDir(tmp.getModValDir(modVal), "credit"))
newFiles = files if force is True else tmp.getFilesByRecency(files, expr)

In [2]:
%autoreload
from dbArtistsParse import dbArtistsRawHTML
from dbArtistsRateYourMusic import dbArtistsRateYourMusic
dbAP = dbArtistsRawHTML(dbArtistsRateYourMusic())
dbAP.parse(expr='< 3 Days')

=========================   rateyourmusic   =========================
Current Time is Fri Sep 24, 2021 09:05 for Parsing Raw HTML Files
Current Time is Fri Sep 24, 2021 09:05 for Finding Files To Parse
Process [Finding Files To Parse] took 24.1 seconds.
Current Time is Fri Sep 24, 2021 09:06 for Parsing 836 Raw HTML Files
Process [Parsing 836 Raw HTML Files] took 0.0 seconds.
Process [Parsing Raw HTML Files] took 24.1 seconds.


In [5]:
%autoreload
from dbArtistsParse import dbArtistsRawFiles
from dbArtistsKWorbiTunes import dbArtistsKWorbiTunes
dbAP = dbArtistsRawFiles(dbArtistsKWorbiTunes(), "data")
dbAP.parse(modVal=0, expr='< 3 Days')

=========================   kworbitunes   =========================
Current Time is Fri Sep 24, 2021 09:27 for Parsing ModVal=0 Credit Files
Current Time is Fri Sep 24, 2021 09:27 for Finding Files To Parse
Process [Finding Files To Parse] took 1.2 minutes.
Current Time is Fri Sep 24, 2021 09:28 for Parsing 2793 New Raw Files
100/2793       Parsing /Volumes/Piggy/Discog/artists-kworbitunes/data/E5B1B1E5B48EE381BEE38195E38288....p 	200/2793       Parsing /Volumes/Piggy/Discog/artists-kworbitunes/data/alanloveday.p 	300/2793       Parsing /Volumes/Piggy/Discog/artists-kworbitunes/data/anuwatsa-nguansakphakdi.p 	400/2793       Parsing /Volumes/Piggy/Discog/artists-kworbitunes/data/bernardpurdie.p 	500/2793       Parsing /Volumes/Piggy/Discog/artists-kworbitunes/data/caburC3A9canela.p 	600/2793       Parsing /Volumes/Piggy/Discog/artists-kworbitunes/data/clarasaraiva.p 	700/2793       Parsing /Volumes/Piggy/Discog/artists-kworbitunes/data/dcappella.p 	800/2793       Parsing /Volumes/Piggy/

In [ ]:
from searchUtils import filesFromDir
from fsUtils import setDir
ffd   = filesFromDir(ext=[".html", ".htm"])
files = ffd.getFiles(setDir(dbAP.disc.getArtistsDir(), "data"))
#newFiles = files if force is True else self.getFilesByRecency(files, expr)

In [ ]:
len(files)

In [ ]:
from recentFilesUtils import recentFiles
rf = recentFiles()
rf.setFiles(files)
newFiles = rf.getFilesByRecency('> 3 Days', debug=True)

In [ ]:
newFiles = dbAP.getFilesByRecency(files, expr='> 3 Days', debug=True)
len(newFiles)

In [ ]:
modValPrimaryFiles = tmp.getArtistPrimaryFiles(99, expr=None, force=True)
dbData = tmp.getDBData(99)

In [ ]:
modValArtistIDs  = {getBaseFilename(ifile): ifile for ifile in modValPrimaryFiles}
missingArtistIDs = [ifile for ifile in modValPrimaryFiles if dbData.get(getBaseFilename(ifile)) is None]
print(len(modValArtistIDs))
print(len(missingArtistIDs))

# Unofficial

In [ ]:
from dbArtistsBase import dbArtistsBase
from fsUtils import isFile
from fileUtils import getBaseFilename
from timeUtils import timestat
from ioUtils import saveFile
from time import sleep
import urllib
from webUtils import getHTML

In [165]:
%autoreload

In [155]:
%autoreload
from dbArtistsDiscogs import dbArtistsDiscogs
dbAP = dbArtistsAssertUnofficial(dbArtistsDiscogs())
dbAP.createUnofficialMetadata(99)
dbAP.downloadMissingArtistUnofficial()

=========================   discogs   =========================
dbArtistsAssertUnofficial(Discogs)
  --> Found 4 Discogs Artist IDs To Ignore


In [180]:
from dbArtistsParse import dbArtistsPrimary
dbAP = dbArtistsPrimary(dbArtistsDiscogs())
dbAP.parse(modVal=99, expr='< 90 Days', force=False)
dbAP

=========================   discogs   =========================
Current Time is Sat Sep 25, 2021 18:31 for Parsing ModVal=99 Primary Files
Current Time is Sat Sep 25, 2021 18:31 for Finding Files To Parse
Process [Finding Files To Parse] took 44.9 seconds.
Current Time is Sat Sep 25, 2021 18:32 for Loading ModVal=99 DB Data
  ===> Forcing Reloads of ModVal=99
Process [Loading ModVal=99 DB Data] took 0.0 seconds.
Current Time is Sat Sep 25, 2021 18:32 for Parsing 12738 New Files For ModVal=99
250/12738      Parsing /Volumes/Piggy/Discog/artists-discogs/99/1088199.p
500/12738      Parsing /Volumes/Piggy/Discog/artists-discogs/99/1175199.p
750/12738      Parsing /Volumes/Piggy/Discog/artists-discogs/99/1263899.p
1000/12738     Parsing /Volumes/Piggy/Discog/artists-discogs/99/1350699.p
1250/12738     Parsing /Volumes/Piggy/Discog/artists-discogs/99/1441699.p
1500/12738     Parsing /Volumes/Piggy/Discog/artists-discogs/99/1533099.p
1750/12738     Parsing /Volumes/Piggy/Discog/artists-discog

True

In [166]:
dbAP = dbArtistsUnofficial(dbArtistsDiscogs())
dbAP.parse(modVal=99, expr='< 90 Days', force=True)

=========================   discogs   =========================
Current Time is Sat Sep 25, 2021 17:49 for Parsing ModVal=99 Unofficial Files
Current Time is Sat Sep 25, 2021 17:49 for Finding Files To Parse
Process [Finding Files To Parse] took 0.0 seconds.
Current Time is Sat Sep 25, 2021 17:49 for Loading ModVal=99 DB Data
  ===> Forcing Reloads of ModVal=99
Process [Loading ModVal=99 DB Data] took 0.0 seconds.
Current Time is Sat Sep 25, 2021 17:49 for Parsing 314 New Unofficial Files For ModVal=99
50/314         Parsing /Volumes/Piggy/Discog/artists-discogs/99/unofficial/148799.p
100/314        Parsing /Volumes/Piggy/Discog/artists-discogs/99/unofficial/211999.p
150/314        Parsing /Volumes/Piggy/Discog/artists-discogs/99/unofficial/269799.p
200/314        Parsing /Volumes/Piggy/Discog/artists-discogs/99/unofficial/356099.p
250/314        Parsing /Volumes/Piggy/Discog/artists-discogs/99/unofficial/545699.p
300/314        Parsing /Volumes/Piggy/Discog/artists-discogs/99/unoffici

In [193]:
%load_ext autoreload
%autoreload
from dbArtistsParse import dbArtistsPrimary
from dbArtistsParseUnofficial import dbArtistsAssertUnofficial, dbArtistsUnofficial
from dbArtistsDiscogs import dbArtistsDiscogs

for modVal in range(100):
    dbAC = dbArtistsAssertUnofficial(dbArtistsDiscogs())
    dbAC.createUnofficialMetadata(modVal=modVal)
    dbAC.downloadMissingArtistUnofficial()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
=========================   discogs   =========================
dbArtistsAssertUnofficial(Discogs)
  --> Found 4 Discogs Artist IDs To Ignore
Current Time is Sat Sep 25, 2021 19:25 for Creating Unofficial Files Metadata
Current Time is Sat Sep 25, 2021 19:25 for Finding Pages/URL/MediaCounts Data For ModVal=99
Process [Finding Pages/URL/MediaCounts Data For ModVal=99] took 3.6 seconds.
Current Time is Sat Sep 25, 2021 19:25 for Finding Artists With Unofficial MediaCounts From 11771 Artists For ModVal=99
Process [Finding Artists With Unofficial MediaCounts From 11771 Artists For ModVal=99] took 0.0 seconds.
Current Time is Sat Sep 25, 2021 19:25 for Removing Ignored Artists From 361 Artists For ModVal=99
Process [Removing Ignored Artists From 361 Artists For ModVal=99] took 0.0 seconds.
Current Time is Sat Sep 25, 2021 19:25 for Finding Known Unofficial Artists From 361 Unofficial Artists For ModVal=

In [194]:
dbAC.downloadMissingArtistUnofficial()

Current Time is Sat Sep 25, 2021 19:25 for Downloading Missing Artist Unofficial Files
Current Time is Sat Sep 25, 2021 19:25 for Downloading 1 Missing Artist Unofficial Files For ModVal=99
0/1:  [Cross & Quinn] / [/artist/4987899-Cross-Quinn]
URL/Savename: [https://www.discogs.com/artist/4987899-Cross-Quinn?sort=year%2Casc&limit=500&type=Unofficial] :: [/Volumes/Piggy/Discog/artists-discogs/99/unofficial/4987899.p]
Error downloading https://www.discogs.com/artist/4987899-Cross-Quinn?sort=year%2Casc&limit=500&type=Unofficial
Process [Downloading 1 Missing Artist Unofficial Files For ModVal=99] took 0.2 seconds.
Process [Downloading Missing Artist Unofficial Files] took 0.2 seconds.


In [192]:
#dbAP = dbArtistsPrimary(dbArtistsDiscogs())
#dbAP.createArtistMetadata(modVal)
#dbAP.createAlbumMetadata(modVal)
%load_ext autoreload
%autoreload
from dbArtistsParse import dbArtistsPrimary
from dbArtistsParseUnofficial import dbArtistsAssertUnofficial, dbArtistsUnofficial
from dbArtistsDiscogs import dbArtistsDiscogs

dbC = dbArtistsUnofficial(dbArtistsDiscogs())
dbC.parse(modVal=modVal, expr='< 1 Days', force=False)
dbC.createArtistMetadata(modVal)
dbC.createAlbumMetadata(modVal)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
=========================   discogs   =========================
Current Time is Sat Sep 25, 2021 19:24 for Parsing ModVal=99 Unofficial Files
Current Time is Sat Sep 25, 2021 19:24 for Finding Files To Parse
Process [Finding Files To Parse] took 0.2 seconds.
Current Time is Sat Sep 25, 2021 19:24 for Loading ModVal=99 DB Data
Process [Loading ModVal=99 DB Data] took 3.7 seconds.
Current Time is Sat Sep 25, 2021 19:24 for Parsing 46 New Unofficial Files For ModVal=99
10/46          Parsing /Volumes/Piggy/Discog/artists-discogs/99/unofficial/1487399.p
20/46          Parsing /Volumes/Piggy/Discog/artists-discogs/99/unofficial/2952099.p
30/46          Parsing /Volumes/Piggy/Discog/artists-discogs/99/unofficial/4698999.p
40/46          Parsing /Volumes/Piggy/Discog/artists-discogs/99/unofficial/664799.p
46/46          Parsing /Volumes/Piggy/Discog/artists-discogs/99/unofficial/986799.p
Process [Parsing 4

In [183]:
len(unofficialData)

361

In [184]:
len(ignoreData)

361

In [185]:
len(unofficialFiles)

314

In [169]:
%load_ext autoreload
%autoreload
from dbArtistsParse import dbArtistsPrimary
from dbArtistsParseUnofficial import dbArtistsAssertUnofficial, dbArtistsUnofficial
from dbArtistsDiscogs import dbArtistsDiscogs

for modVal in range(98,99):
    dbAP = dbArtistsPrimary(dbArtistsDiscogs())
    _ = dbAP.parse(modVal=modVal, expr='< 0 Days', force=False)
    dbAP.createArtistMetadata(modVal)
    dbAP.createAlbumMetadata(modVal)

    dbAC = dbArtistsAssertUnofficial(dbArtistsDiscogs())
    dbAC.createUnofficialMetadata(modVal=modVal)
    dbAC.downloadMissingArtistUnofficial()

    dbC = dbArtistsUnofficial(dbArtistsDiscogs())
    dbC.parse(modVal=modVal, expr='< 90 Days', force=True)
    dbC.createArtistMetadata(modVal)
    dbC.createAlbumMetadata(modVal)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
=========================   discogs   =========================
Current Time is Sat Sep 25, 2021 18:06 for Parsing ModVal=98 Primary Files
Current Time is Sat Sep 25, 2021 18:06 for Finding Files To Parse
Process [Finding Files To Parse] took 2.0 minutes.
Current Time is Sat Sep 25, 2021 18:08 for Parsing 0 New Files For ModVal=98
Process [Parsing 0 New Files For ModVal=98] took 0.0 seconds.
Process [Parsing ModVal=98 Primary Files] took 2.0 minutes.
Current Time is Sat Sep 25, 2021 18:08 for Creating Artist Metadata For ModVal=98
Saving 319 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-discogs-db/metadata/98-Metadata.p
Process [Creating Artist Metadata For ModVal=98] took 0.2 seconds.
Current Time is Sat Sep 25, 2021 18:08 for Creating Artist Album Metadata For ModVal=98
Saving 319 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-discogs-db/metadata/98-MediaMetada

# Extra

In [111]:

from dbArtistsMusicBrainz import dbArtistsMusicBrainz
dbAP = dbArtistsAssertExtra(dbArtistsMusicBrainz())

NameError: name 'dbArtistsAssertUnofficial' is not defined

In [108]:
%load_ext autoreload
%autoreload
from dbArtistsParse import dbArtistsPrimary
from dbArtistsParseExtra import dbArtistsExtra, dbArtistsAssertExtra
from dbArtistsMusicBrainz import dbArtistsMusicBrainz

for modVal in range(72,73):
    if False:
        dbAP = dbArtistsPrimary(dbArtistsMusicBrainz())
        _ = dbAP.parse(modVal=modVal, expr='< 0 Days', force=False)
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)
        continue

    if False:
        dbAC = dbArtistsAssertExtra(dbArtistsMusicBrainz())
        dbAC.createExtraMetadata(modVal=modVal)
        dbAC.downloadMissingArtistExtras()
    
    if True:
        dbC = dbArtistsExtra(dbArtistsMusicBrainz())
        dbC.parse(modVal=modVal, expr='< 90 Days', force=True)
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
=========================   musicbrainz   =========================
Current Time is Sat Sep 25, 2021 15:35 for Parsing ModVal=72 Extra Files
Current Time is Sat Sep 25, 2021 15:35 for Finding Files To Parse
Process [Finding Files To Parse] took 0.0 seconds.
Current Time is Sat Sep 25, 2021 15:35 for Loading ModVal=72 DB Data
Process [Loading ModVal=72 DB Data] took 4.3 seconds.
Current Time is Sat Sep 25, 2021 15:35 for Parsing 52 New Extra Files For ModVal=72
10/52          Parsing /Volumes/Piggy/Discog/artists-musicbrainz/72/extra/46011490395709286262662846329696254272-17.p
20/52          Parsing /Volumes/Piggy/Discog/artists-musicbrainz/72/extra/46011490395709286262662846329696254272-26.p
30/52          Parsing /Volumes/Piggy/Discog/artists-musicbrainz/72/extra/46011490395709286262662846329696254272-35.p
40/52          Parsing /Volumes/Piggy/Discog/artists-musicbrainz/72/extra/4601149039570928626

In [110]:
%load_ext autoreload
%autoreload
from dbArtistsParse import dbArtistsPrimary
from dbArtistsParseExtra import dbArtistsExtra, dbArtistsAssertExtra
from dbArtistsDiscogs import dbArtistsDiscogs

for modVal in range(72,73):
    if False:
        dbAP = dbArtistsPrimary(dbArtistsDiscogs())
        _ = dbAP.parse(modVal=modVal, expr='< 0 Days', force=False)
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

    if True:
        dbAC = dbArtistsAssertExtra(dbArtistsDiscogs())
        dbAC.createExtraMetadata(modVal=modVal)
        dbAC.downloadMissingArtistExtras()

        dbC = dbArtistsExtra(dbArtistsDiscogs())
        dbC.parse(modVal=modVal, expr='< 90 Days', force=True)
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
=========================   discogs   =========================
dbArtistsAssertExtra(Discogs)
  --> Found 4 AllMusic Song IDs To Ignore
Current Time is Sat Sep 25, 2021 15:49 for Creating Extra Files Metadata
Current Time is Sat Sep 25, 2021 15:49 for Finding Pages/URL Data For ModVal=72
Process [Finding Pages/URL Data For ModVal=72] took 4.5 seconds.
Current Time is Sat Sep 25, 2021 15:49 for Finding Artists With More Pages From 11762 Artists For ModVal=72
Process [Finding Artists With More Pages From 11762 Artists For ModVal=72] took 0.0 seconds.
Current Time is Sat Sep 25, 2021 15:49 for Removing Ignored Artists From 78 Artists For ModVal=72
Process [Removing Ignored Artists From 78 Artists For ModVal=72] took 0.0 seconds.
Current Time is Sat Sep 25, 2021 15:49 for Saving Metadata From 78/78/11762 For ModVal=72
Process [Saving Metadata From 78/78/11762 For ModVal=72] took 0.0 seconds.
Process [Cr

74/78:  [John Taylor (15)] / [/artist/853272-John-Taylor-15] / [3-4]
75/78:  [Erwin Wieringa] / [/artist/915372-Erwin-Wieringa]
76/78:  [Timothy Kraemer] / [/artist/961372-Timothy-Kraemer]
77/78:  [William Hayman Cummings] / [/artist/992372-William-Hayman-Cummings]
Process [Downloading 78 Missing Artist Extra Files For ModVal=72] took 0.4 seconds.
Process [Downloading Missing Artist Extra Files] took 0.4 seconds.
=========================   discogs   =========================
Current Time is Sat Sep 25, 2021 15:49 for Parsing ModVal=72 Extra Files
Current Time is Sat Sep 25, 2021 15:49 for Finding Files To Parse
Process [Finding Files To Parse] took 0.2 seconds.
Current Time is Sat Sep 25, 2021 15:49 for Loading ModVal=72 DB Data
Process [Loading ModVal=72 DB Data] took 5.7 seconds.
Current Time is Sat Sep 25, 2021 15:49 for Parsing 13 New Extra Files For ModVal=72
10/13          Parsing /Volumes/Piggy/Discog/artists-discogs/72/extra/736972-2.p
13/13          Parsing /Volumes/Piggy/Dis

In [100]:
%autoreload
from dbArtistsMusicBrainz import dbArtistsMusicBrainz
dbC = dbArtistsExtra(dbArtistsMusicBrainz())
dbC.parse(modVal=80, expr='< 90 Days', force=True)
dbAP.createArtistMetadata(80)
dbAP.createAlbumMetadata(80)

=========================   musicbrainz   =========================
Current Time is Sat Sep 25, 2021 15:24 for Parsing ModVal=80 Extra Files
Current Time is Sat Sep 25, 2021 15:24 for Finding Files To Parse
Process [Finding Files To Parse] took 0.0 seconds.
Current Time is Sat Sep 25, 2021 15:24 for Loading ModVal=80 DB Data
Process [Loading ModVal=80 DB Data] took 3.7 seconds.
Current Time is Sat Sep 25, 2021 15:24 for Parsing 8 New Extra Files For ModVal=80
8/8            Parsing /Volumes/Piggy/Discog/artists-musicbrainz/80/extra/79157337509398256567626992412392412480-2.p
Process [Parsing 8 New Extra Files For ModVal=80] took 0.9 seconds.
Saving 8 new artist IDs to /Users/tgadfort/Music/Discog/artists-musicbrainz-db/80-DB.p
Saving 8383 total artist IDs to /Users/tgadfort/Music/Discog/artists-musicbrainz-db/80-DB.p
Saving 21636 total artist media
Current Time is Sat Sep 25, 2021 15:24 for Creating Artist Metadata For ModVal=80
Saving 8383 new artist IDs name data to /Users/tgadfort/Mu

In [82]:
dbAP.createExtraMetadata(72)

Current Time is Sat Sep 25, 2021 13:44 for Creating Extra Files Metadata
Current Time is Sat Sep 25, 2021 13:44 for Finding Pages/URL Data For ModVal=72
Process [Finding Pages/URL Data For ModVal=72] took 4.2 seconds.
Current Time is Sat Sep 25, 2021 13:44 for Finding Artists With More Pages From 8584 Artists For ModVal=72
Process [Finding Artists With More Pages From 8584 Artists For ModVal=72] took 0.0 seconds.
Current Time is Sat Sep 25, 2021 13:44 for Removing Ignored Artists From 5 Artists For ModVal=72
Process [Removing Ignored Artists From 5 Artists For ModVal=72] took 0.0 seconds.
Current Time is Sat Sep 25, 2021 13:44 for Saving Metadata From 4/5/8584 For ModVal=72
Process [Saving Metadata From 4/5/8584 For ModVal=72] took 0.0 seconds.
Process [Creating Extra Files Metadata] took 4.2 seconds.


In [65]:
from copy import deepcopy
meta = deepcopy(dbAP.metadata)

In [76]:
dbAP.downloadMissingArtistExtras()

Current Time is Sat Sep 25, 2021 13:43 for Downloading Missing Artist Extra Files
Current Time is Sat Sep 25, 2021 13:43 for Downloading 4 Missing Artist Extra Files For ModVal=72
0/4:  [Paul Mauriat] / [/artist/87496c04-c857-4bd5-b86b-345129a1be63]
1/4:  [Miklós Rózsa] / [/artist/0008af7d-2aa1-4b4d-80af-b3b64ee3cac6]
2/4:  [Vijay Prakash] / [/artist/acd91eff-bdbb-4c0b-b5b4-664d76c1f14e]
3/4:  [ASC] / [/artist/f64c9779-13d0-4f8f-b6ba-c9de929c4a23]
Process [Downloading 4 Missing Artist Extra Files For ModVal=72] took 0.0 seconds.
Process [Downloading Missing Artist Extra Files] took 0.0 seconds.


In [49]:
ignoreData = dbAP.getMasterIgnoreData()

In [50]:
ignoreData

{'AllMusic': {'Credit': ['0003664630',
   '0000043332',
   '002418837',
   '0001654631',
   '0002710024',
   '0003245434',
   '0001634699',
   '0001544204',
   '0002879590',
   '0001610403',
   '0002871708',
   '0000111404',
   '0000445110',
   '0003480802',
   '0002031919',
   '0001596722',
   '0003913619',
   '0000932904',
   '0003861632',
   '0001541219',
   '0002323590',
   '0001992534',
   '0003622807',
   '0000684314',
   '0002501135',
   '0003184911',
   '0001184437',
   '0002050837',
   '0001357202',
   '0000464822',
   '0002948225',
   '0001632428',
   '0003064235',
   '0001439486',
   '0002099506',
   '0002107008',
   '0000120615',
   '0002957086',
   '0003520225',
   '0001763101',
   '0000511318',
   '0001328319',
   '0003895810',
   '0001753529',
   '0003105717',
   '0002339670',
   '0001622804',
   '0003409909',
   '0000390208',
   '0003975018',
   '0001780324',
   '0000390317',
   '0003299308',
   '0001980017',
   '0002077218',
   '0001185737',
   '0000437170',
   '000261

In [51]:
def getIgnoreArtistList():

    ###########################################################################################################################
    # Ignore List
    ############################################################################################################################
    ignoreArtistList = {}
    ignoreArtistList["Discogs"]  = ["M", "Bernard Edwards & Nile Rodgers", "Pomus-Shuman", "Tepper-Bennett"]
    ignoreArtistList["LastFM"]   = ["MAX", "Sonny Boy Williamson", "Trouble", "NRG", "Olivia", "Hunter", "Zeus", "Cobra", 
                                    "Shogun", "Nomad", "Juice", "Tyrant", "Willow", "Airbag", "Deuce", "Budgie", "Eclipse",
                                    "Jumbo", "L.E.J.", "Sniper", "Ceremony", "Voyager", "Solange", "Logos", 
                                    "The Highwaymen", "Rose", "Blake", "Tash", "Cast", "Friends", "Collage", "Ripe", 
                                    "Undercover", "Quest", "Holden"]

    ignoreArtistList["LastFM"]  += ["Íåèçâåñòíûé Èñïîëíèòåëü", "Coast To Coast AM - George Noory", 
                                    "[unknown]", "Various Composers", "[unknown]", "Zåìôèðà", 'Nature Sounds']
    ignoreArtistList["AllMusic"] = ["Bryan Adams", "Tina"]
    ignoreArtistList["MusicBrainz"] = ["Various Artists", "[unknown]", "[no artist]", "[language instruction]",
                                      "[nature sounds]", "Die drei ???", "[dialogue]", "初音ミク", '[data]', 
                                       'Geovanni', "Juan Torres", "DrefQuila", '[Disney]', 
                                       '[Christmas music]', 'Harry Nach', 'Tiago PZK']
    ignoreArtistList["RateYourMusic"] = []
    ignoreArtistList["Genius"] = []
    ignoreArtistList["KWorbiTunes"] = []
    ignoreArtistList["KWorbSpotify"] = []
    ignoreArtistList["Deezer"] = []
    ignoreArtistList["AlbumOfTheYear"] = []
    return ignoreArtistList

In [55]:
for db,artistNames in getIgnoreArtistList().items():
    dbAP.updateMasterIgnoreNameData(db,artistNames)

Found 4 Name Ignores
Adding 4 Name Ignores
Now Have 4 Name Ignores
Found 41 Name Ignores
Adding 42 Name Ignores
Now Have 41 Name Ignores
Found 2 Name Ignores
Adding 2 Name Ignores
Now Have 2 Name Ignores
Found 16 Name Ignores
Adding 16 Name Ignores
Now Have 16 Name Ignores
Found 0 Name Ignores
Adding 0 Name Ignores
Now Have 0 Name Ignores
Found 0 Name Ignores
Adding 0 Name Ignores
Now Have 0 Name Ignores
Found 0 Name Ignores
Adding 0 Name Ignores
Now Have 0 Name Ignores
Found 0 Name Ignores
Adding 0 Name Ignores
Now Have 0 Name Ignores
Found 0 Name Ignores
Adding 0 Name Ignores
Now Have 0 Name Ignores
Found 0 Name Ignores
Adding 0 Name Ignores
Now Have 0 Name Ignores
